# Motivation

In [Novikov eta al. 2016](http://papers.nips.cc/paper/5787-tensorizing-neural-networks.pdf) they use the tensor-train representation to construct a weight matrix. However, the tensor-train constructs a high dimensional teensor and they simply reshape it into a matrix. I thought this was interesting/weird and want to investigate. 

Specifically, I am interested in how parameters are shared across the constructed weight matrix. Weight tying is an important part of designing neural networks, and I am interested in the relationship between parameter tying schemes and tensor (networks) and reshaping.

The motivating example would be that a convolution can be written as a parameter sharing scheme in matrix form. Constructed using a circulant, ...?!

***

Secondly, when looking at the [algol for HOSVD](https://lirias.kuleuven.be/bitstream/123456789/72517/1/94-31.pdf) there is an unfolding (aka reshape) operation that is used to matricse the tensors so the left singular vectors of each dimension can be calculated.

### The tensor train format (aka MPS)



In [1]:
import functools

import sympy as sym
sym.init_printing(use_latex='mathjax')
import numpy as np
import matplotlib.pyplot as plt

# NEED a way to visualise!

# dont want to just focus on TT format.
# what other interesting ones are there? 
# Have a look at MPS and PEPS?
# something more exotic?

In [92]:
idx = 'abcdefghijkl'  # names for indexes

def construct_core(idx, n):
    # construct a 3-tensor
    return sym.tensor.Array([[[sym.Symbol('{}_{}{}{}'.format(idx,i,j,k)) 
                               for i in range(n)] 
                              for j in range(n)] 
                             for k in range(n)])
    
def construct_cores(N, n):
    return [construct_core(idx[i], n) for i in range(N)]

def construct_tensor(cores):
    t = cores[0]
    for i in range(len(cores)-1):
        t = sym.tensorproduct(t, cores[i+1])
        t = sym.tensorcontraction(t, (3,4))  # not sure if this is right...
    return t

In [95]:
n_cores = 2
cores = construct_cores(n_cores, 2)
t = construct_tensor(cores)
print(t.shape)

t = sym.simplify(t)
t = sym.reshape(t, [8])

(2, 2, 2, 2)


In [96]:
t

⎡a₀₀₀⋅(b₀₀₀ + b₀₁₁)  a₀₀₀⋅(b₁₀₀ + b₁₁₁)  a₁₀₀⋅(b₀₀₀ + b₀₁₁)  a₁₀₀⋅(b₁₀₀ + b₁₁₁
⎢                                                                             
⎣a₀₀₁⋅(b₀₀₀ + b₀₁₁)  a₀₀₁⋅(b₁₀₀ + b₁₁₁)  a₁₀₁⋅(b₀₀₀ + b₀₁₁)  a₁₀₁⋅(b₁₀₀ + b₁₁₁

)  a₀₁₀⋅(b₀₀₀ + b₀₁₁)  a₀₁₀⋅(b₁₀₀ + b₁₁₁)  a₁₁₀⋅(b₀₀₀ + b₀₁₁)  a₁₁₀⋅(b₁₀₀ + b₁
                                                                              
)  a₀₁₁⋅(b₀₀₀ + b₀₁₁)  a₀₁₁⋅(b₁₀₀ + b₁₁₁)  a₁₁₁⋅(b₀₀₀ + b₀₁₁)  a₁₁₁⋅(b₁₀₀ + b₁

₁₁)⎤
   ⎥
₁₁)⎦

In [ ]:
# can even cosntruct tensors where we have the same 
# or more parameters than elements, but they are shared
# in interesting ways

In [93]:
s = str(t) 
print(s.count('a_000'))
print(s.count('a_001'))

# so each parameter is shared over eight spaces?
# want a general formula for this

# also there is an a_{}{}{} parameter in every element.

# kind of locality prior?
# each parameter is shared over some local set (a row or colum).

8
8


### The various forms of tensor SVD

So, what about all the reshaping funny business going on in HSVD and HOSVD?



In [59]:
def unfold(tensor, axis): # aka Mode, matricization, fibre
    return np.reshape(tensor, (tensor.shape[axis], -1))

I do not have any good intuition for why/how taking the
left eigenvectors of a reshaped tensor ...?
so somehow, under reshaping, the left singular vectors are preserved?
S and V are unncessary (which seems rather unusual...)

***
The way the core singular value tensor is calucluated seems like cheating.
$$
\mathcal A = S \times_1 U_1 ... \times_n U_n \\
S = \mathcal A \times_1 U_1^T ... \times_n U_n^T \\
$$
this doesnt seem right, S should be diagonal!?

***

Hierarchical SVD also uses the same trick.
Should I bother coding it?
Seems interesting as now you have multiple core tensors and they need to be reconstructed using the right graph.

In [73]:
class HOSVD():
    def decompose(tensor):
        U = []
        # for each arm of the tensor
        for i, s in enumerate(tensor.shape):
            u, _, _ = np.linalg.svd(unfold(tensor, i))
            U.append(u)

        S = tensor
        for i, leg in enumerate(U):
            S = np.tensordot(leg.T, S, axes=[1, i])
        return U, S

    def construct(legs, core):
        c = core
        # or could outerproduct the legs first and then elementwise mul!?
        for i, leg in enumerate(legs):
            c = np.tensordot(leg, c, axes=[1, i])
        return c
    
    def test():
        A = np.random.random((5,5,5))
        u, s = HOSVD.decompose(A)
        B = HOSVD.construct(u ,s)
        d = np.sum(np.abs(A - B))
        if d > 1e-8:
            raise ValueError('A and B are not equal. Difference = {}'.format(d))
            
HOSVD.test()

Ok, so that is the motivation out of the way... phew. Now lets take a closer look at reshaping.

Main questions;

...

## Reshaping algols

* Row first, outer vs inner.
* Can just be done at read time with different striding patterns (see views in numpy?)

Only real requirements are that is must have an inverse? It is consistent? It ...?
What about more 'random' permutations on the indexes?

What if we thought about it as a function? What does a reshape do?
Have some $f: x->y$ but we change the f while preserving XXX?!? What is preserved? What are its symmetries?  

In [ ]:
def reshape(tensor):
    pass

Is reshaing a linear op!?
Does it commute, associate, distribute, ...
Firstly, its a unary operation?! So not sure what to do with that...

### Associativity

$\varrho(u) + (v + w) = (\varrho(u) + v) + w$

### Commutativity

$\varrho(a) + b = b + \varrho(a)$


$a(\mathring u + v) = \mathring{au} + av$



Reshaping is a permutation of the bases?



Reshape.
Want;
- some properties that I can measure!?!
- some visualisations! (what happens when I reshape?)
- better intuition... need a concrete example to play with
-

#### Neighborhoods

Picture I already have. Neighbors and where they go to.


#### Connectedness (the dual of neighborhoods?)

What about the graph POV?

#### How is reshape like a convolution?

For example, this is what we do when we want to do a convolution. Construct a tensor of patches (examples, X, Y, kernel, kernel) and then reshape it into a (examples x X x Y, kernel x kernel ) matrix.



## Parameter sharing

What is it, why do we do it, some examples.
Can represent large(r) spaces with fewer parameters (that is the usual argument for TNs on parameter sharing...)

Sharing over;

* space,
* time,
* relations,
* ?

Nice way to build priors about invariance. (how does this related to the structure of tensor networks!?)

Aka, parameter sharing schemes. If we write the reshaped, constructed tensor, and show the receptive field of original parameters.
- are the receptive fields local, which tensor-nets/reshapings give local receptive fields?
- ?
-

This idea is orthogonal to reshaping, reshaping is just a nice way to visualise it?


$$\begin{aligned}
&= \begin{bmatrix}
a_{11} & a_{12} & a_{13} & a_{14} & a_{15} & a_{16} \\
a_{21} & a_{22} & a_{23} & a_{24} & a_{25} & a_{26} \\
a_{31} & a_{32} & a_{33} & a_{34} & a_{35} & a_{36} \\
a_{41} & a_{42} & a_{43} & a_{44} & a_{45} & a_{46} \\
a_{51} & a_{52} & a_{53} & a_{54} & a_{55} & a_{56} \\
a_{61} & a_{62} & a_{63} & a_{64} & a_{65} & a_{66} \\
\end{bmatrix} \\
&\text{(stack by columns. reshape by first indices fastest)}\\
&= \begin{bmatrix}
\begin{bmatrix}
a_{11} &  a_{31} & a_{51}\\
a_{21} & a_{41} & a_{61}\\
\end{bmatrix} & 
\begin{bmatrix}
a_{12} &  a_{32} & a_{52}\\
a_{22} & a_{42} & a_{62}\\
\end{bmatrix}\\
\begin{bmatrix}
a_{13} &  a_{33} & a_{53}\\
a_{23} & a_{43} & a_{63}\\
\end{bmatrix} & 
\begin{bmatrix}
a_{14} &  a_{34} & a_{54}\\
a_{24} & a_{44} & a_{64}\\
\end{bmatrix} \\
\begin{bmatrix}
a_{15} &  a_{35} & a_{55}\\
a_{25} & a_{45} & a_{65}\\
\end{bmatrix} & 
\begin{bmatrix}
a_{16} &  a_{36} & a_{56}\\
a_{26} & a_{46} & a_{66}\\
\end{bmatrix} \\
\end{bmatrix}\end{aligned}$$

Distances are not preserved. Originally $a_{33}$ is one index away from
$a_{32},a_{34},a_{23},a_{43}$. But after the reshaping, the set of
elements that d=1 are $a_{13},a_{53},a_{43},a_{31},a_{35},a_{34}$.
If we map these back into the original matrix, we can see that the
‘range’ of the indicies is speading. More are in each elements
neighbourhood. What does this mean?
